# Step 1 - Frame Dividing Part

#### Install Opencv for Frame Extraction

In [1]:
pip install opencv-python-headless

  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.7 MB 7.4 MB/s eta 0:00:02
   ---------- ----------------------------- 3.4/12.7 MB 10.3 MB/s eta 0:00:01
   ------------------ --------------------- 6.0/12.7 MB 11.0 MB/s eta 0:00:01
   --------------------------- ------------ 8.7/12.7 MB 11.6 MB/s eta 0:00:01
   ---------------------------------- ----- 11.0/12.7 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 12.7/12.7 MB 11.3 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   --------

#### Importing Libraries CV2 and OS

In [2]:
import cv2
import os

#### Function to Extract Frames

In [10]:
import os
import cv2

def extract_frames(video_path, output_folder, frame_rate=5, resize=(224, 224)):
    """Extract resized frames from a video and save to output_folder"""
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    interval = max(1, int(fps / frame_rate))

    count = 0
    frame_id = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % interval == 0:
            resized = cv2.resize(frame, resize)
            frame_path = os.path.join(output_folder, f"frame_{frame_id:04d}.jpg")
            cv2.imwrite(frame_path, resized)
            frame_id += 1
        count += 1
    cap.release()
    print(f"Extracted {frame_id} frames from {video_path} → {output_folder}")

def process_all_videos(input_root, output_root, frame_rate=5):
    """Walk through input_root, find all .mp4/.mov, and extract frames into a folder named after each video"""
    for root, _, files in os.walk(input_root):
        for file in files:
            if file.endswith(('.mp4', '.MOV')):
                video_path = os.path.join(root, file)
                
                # Get relative path from input_root to current set folder
                rel_folder = os.path.relpath(root, input_root)
                
                # Output folder is: output_root / relative_folder / video_name_without_ext
                video_name = os.path.splitext(file)[0]
                output_folder = os.path.join(output_root, rel_folder, video_name)

                extract_frames(video_path, output_folder, frame_rate)




In [ ]:


# Example usage
input_root = "Input"
output_root = "Output"
process_all_videos(input_root, output_root, frame_rate=5)

Extracted 35 frames from data\CP\Player02_CP_Set_01\Player02_CP_60Kg_front_view.MOV → output\CP\Player02_CP_Set_01\Player02_CP_60Kg_front_view
Extracted 31 frames from data\CP\Player02_CP_Set_01\Player02_CP_60Kg_side_view.MOV → output\CP\Player02_CP_Set_01\Player02_CP_60Kg_side_view
Extracted 34 frames from data\CP\Player02_CP_Set_01\Player02_CP_60Kg_top_view.mp4 → output\CP\Player02_CP_Set_01\Player02_CP_60Kg_top_view
Extracted 36 frames from data\CP\Player02_CP_Set_02\Player02_CP_80Kg_front_view.MOV → output\CP\Player02_CP_Set_02\Player02_CP_80Kg_front_view


KeyboardInterrupt: 